In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "69b6fe2c",
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import pandas as pd\n",
    "import seaborn as sns\n",
    "\n",
    "import matplotlib\n",
    "import matplotlib.pyplot as plt\n",
    "from matplotlib.lines import Line2D\n",
    "from matplotlib.patches import Patch\n",
    "from matplotlib.ticker import MaxNLocator\n",
    "import matplotlib.patches as mpatches\n",
    "\n",
    "matplotlib.rcParams['text.usetex'] = True\n",
    "# matplotlib.rcParams['text.latex.preview'] = True\n",
    "plt.rc('font', family='serif', serif=['Times'])\n",
    "\n",
    "cm = plt.get_cmap('Set3') \n",
    "colors = cm.colors"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "dbc322fa",
   "metadata": {},
   "outputs": [],
   "source": [
    "models = [\n",
    "    '$\\\\mathrm{\\\\textsc{LXMERT}_{180K}}$', #0\n",
    "    '$\\\\mathrm{\\\\textsc{LXMERT}_{180K}^{N}}$', #1\n",
    "    '$\\\\mathrm{\\\\textsc{LXMERT}_{3M}}$', #2\n",
    "    '$\\\\mathrm{\\\\textsc{LXMERT}_{3M}^{N}}$', #3\n",
    "    '$\\\\mathrm{\\\\textsc{ALBEF}_{4M}}$', #4\n",
    "    '$\\\\mathrm{\\\\textsc{ALBEF}_{4M}^{N-COCO}}$', #5\n",
    "    '$\\\\mathrm{\\\\textsc{ALBEF}_{4M}^{N-CC3M}}$', #6\n",
    "    '$\\\\mathrm{\\\\textsc{ALBEF}_{14M}}$', #7\n",
    "    '$\\\\mathrm{\\\\textsc{ALBEF}_{14M}^{N-COCO}}$', #8\n",
    "    '$\\\\mathrm{\\\\textsc{ALBEF}_{14M}^{N-CC3M}}$', #9\n",
    "    '$\\\\mathrm{\\\\textsc{BLIP}_{129M}}$', #10\n",
    "    '$\\\\mathrm{\\\\textsc{BLIP}_{129M}^{N}}$' #11\n",
    "]"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "eb40ef62",
   "metadata": {},
   "source": [
    "# VQAv2"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "069d681c",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x432 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "# Define the data \n",
    "lxmert = {\"M\": 77.5, \"F\": 76.3, \"N\": 72.7}\n",
    "lxmert_n = {\"M\": 79.6, \"F\":78.1, \"N\":79.3}\n",
    "concaplxmert = {\"M\":68.4, \"F\":66.5, \"N\":62.4}\n",
    "concaplxmert_n = {\"M\":70.3, \"F\":67.9, \"N\":70.1}\n",
    "albef14m = {\"M\":76.0, \"F\":75.0, \"N\":75.6}\n",
    "albef14m_n = {\"M\":76.0, \"F\":74.9, \"N\":75.5}\n",
    "concapalbef14m_n = {\"M\":75.8, \"F\":75.0, \"N\":75.5}\n",
    "blip = {\"M\":76.5, \"F\":75.1, \"N\":75.6}\n",
    "blip_n = {\"M\":76.4, \"F\":75.2, \"N\":75.8}\n",
    "\n",
    "# Plot\n",
    "fig = plt.figure(figsize=(6,6), tight_layout=True)\n",
    "ax = fig.add_subplot()\n",
    "\n",
    "bcm = plt.get_cmap('Pastel2')\n",
    "bcolors = bcm.colors\n",
    "ax.set_xlim(-0.5, 2.5)\n",
    "ax.axvspan(-0.5, 0.5, facecolor=bcolors[0], alpha=0.2)\n",
    "ax.axvspan(0.5, 1.5, facecolor=bcolors[1], alpha=0.3)\n",
    "ax.axvspan(1.5, 2.5, facecolor=bcolors[2], alpha=0.2)\n",
    "\n",
    "w = 0.25\n",
    "size=14\n",
    "j = 0.1\n",
    "# LXMERT\n",
    "i = 0\n",
    "ax.plot(i-w + np.random.rand(1)[0]*j, lxmert[\"M\"], 'o', markersize=size, markeredgecolor='k', color=colors[0])\n",
    "ax.plot(i-w + np.random.rand(1)[0]*j, lxmert[\"F\"], 'o', markersize=size, markeredgecolor='k', color=colors[3])\n",
    "ax.plot(i-w + np.random.rand(1)[0]*j, lxmert[\"N\"], 'o', markersize=size, markeredgecolor='k', color=colors[1])\n",
    "ax.plot(i+w + np.random.rand(1)[0]*j, lxmert_n[\"M\"], 's', markersize=size, markeredgecolor='k', color=colors[0])\n",
    "ax.plot(i+w + np.random.rand(1)[0]*j, lxmert_n[\"F\"], 's', markersize=size, markeredgecolor='k', color=colors[3])\n",
    "ax.plot(i+w + np.random.rand(1)[0]*j, lxmert_n[\"N\"], 's', markersize=size, markeredgecolor='k', color=colors[1])\n",
    "\n",
    "# ALBEF14m\n",
    "i = 1\n",
    "ax.plot(i-w + np.random.rand(1)[0]*j, albef14m[\"M\"], 'o', markersize=size, markeredgecolor='k', color=colors[0])\n",
    "ax.plot(i-w + np.random.rand(1)[0]*j, albef14m[\"F\"], 'o', markersize=size, markeredgecolor='k', color=colors[3])\n",
    "ax.plot(i-w + np.random.rand(1)[0]*j, albef14m[\"N\"], 'o', markersize=size, markeredgecolor='k', color=colors[1])\n",
    "ax.plot(i + np.random.rand(1)[0]*j, albef14m_n[\"M\"], 's', markersize=size, markeredgecolor='k', color=colors[0])\n",
    "ax.plot(i + np.random.rand(1)[0]*j, albef14m_n[\"F\"], 's', markersize=size, markeredgecolor='k', color=colors[3])\n",
    "ax.plot(i + np.random.rand(1)[0]*j, albef14m_n[\"N\"], 's', markersize=size, markeredgecolor='k', color=colors[1])\n",
    "ax.plot(i+w + np.random.rand(1)[0]*j, concapalbef14m_n[\"M\"], 'D', markersize=size, markeredgecolor='k', color=colors[0])\n",
    "ax.plot(i+w + np.random.rand(1)[0]*j, concapalbef14m_n[\"F\"], 'D', markersize=size, markeredgecolor='k', color=colors[3])\n",
    "ax.plot(i+w + np.random.rand(1)[0]*j, concapalbef14m_n[\"N\"], 'D', markersize=size, markeredgecolor='k', color=colors[1])\n",
    "\n",
    "\n",
    "# BLIP\n",
    "i = 2\n",
    "ax.plot(i-w + np.random.rand(1)[0]*j, blip[\"M\"], 'o', markersize=size, markeredgecolor='k', color=colors[0])\n",
    "ax.plot(i-w + np.random.rand(1)[0]*j, blip[\"F\"], 'o', markersize=size, markeredgecolor='k', color=colors[3])\n",
    "ax.plot(i-w + np.random.rand(1)[0]*j, blip[\"N\"], 'o', markersize=size, markeredgecolor='k', color=colors[1])\n",
    "ax.plot(i+w + np.random.rand(1)[0]*j, blip_n[\"M\"], 's', markersize=size, markeredgecolor='k', color=colors[0])\n",
    "ax.plot(i+w + np.random.rand(1)[0]*j, blip_n[\"F\"], 's', markersize=size, markeredgecolor='k', color=colors[3])\n",
    "ax.plot(i+w + np.random.rand(1)[0]*j, blip_n[\"N\"], 's', markersize=size, markeredgecolor='k', color=colors[1])\n",
    "\n",
    "\n",
    "ax.set_ylim(70, 81)\n",
    "ax.set_xticks([x for x in range(3)])\n",
    "ax.set_xticklabels([models[0], models[7], models[10]], fontsize=20)\n",
    "ax.tick_params(axis=\"y\", labelsize=18)\n",
    "# ax.tick_params(axis='x', which='major', horizontalalignment=3)\n",
    "ax.grid(color='gray', alpha=0.2)\n",
    "\n",
    "h = [Patch(facecolor=colors[0], edgecolor='k', label='male'),\n",
    "     Patch(facecolor=colors[3], edgecolor='k', label='female'),\n",
    "     Patch(facecolor=colors[1], edgecolor='k', label='neutral')]\n",
    "leg2 = ax.legend(handles=h, ncol=1, scatterpoints=1, fontsize=16, loc='lower right')\n",
    "# ax.add_artist(leg1)\n",
    "\n",
    "# plt.savefig('figures/vqa_selection_gender.pdf', bbox_extra_artists=(leg1,), bbox_inches='tight')\n",
    "plt.savefig('figures/vqa_selection_gender_coco.pdf', bbox_inches='tight')"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "b295cdfe",
   "metadata": {},
   "source": [
    "# VQAv2 DBA"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "7fd72bc9",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>M</th>\n",
       "      <th>F</th>\n",
       "      <th>N</th>\n",
       "      <th>model</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>lxmert</th>\n",
       "      <td>-0.029297</td>\n",
       "      <td>-0.068376</td>\n",
       "      <td>0.004679</td>\n",
       "      <td>lxmert</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>lxmert_n</th>\n",
       "      <td>-0.019950</td>\n",
       "      <td>-0.051893</td>\n",
       "      <td>0.000668</td>\n",
       "      <td>lxmert_n</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>concaplxmert</th>\n",
       "      <td>-0.020508</td>\n",
       "      <td>-0.064713</td>\n",
       "      <td>-0.037433</td>\n",
       "      <td>concaplxmert</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>concaplxmert_n</th>\n",
       "      <td>-0.020787</td>\n",
       "      <td>-0.074481</td>\n",
       "      <td>-0.023619</td>\n",
       "      <td>concaplxmert_n</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>albef4m</th>\n",
       "      <td>-0.020787</td>\n",
       "      <td>-0.068071</td>\n",
       "      <td>0.000446</td>\n",
       "      <td>albef4m</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>albef4m_n</th>\n",
       "      <td>-0.025530</td>\n",
       "      <td>-0.074786</td>\n",
       "      <td>-0.006907</td>\n",
       "      <td>albef4m_n</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>concapalbef4m_n</th>\n",
       "      <td>-0.023996</td>\n",
       "      <td>-0.065934</td>\n",
       "      <td>-0.008467</td>\n",
       "      <td>concapalbef4m_n</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>albef14m</th>\n",
       "      <td>-0.034738</td>\n",
       "      <td>-0.074786</td>\n",
       "      <td>0.000446</td>\n",
       "      <td>albef14m</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>albef14m_n</th>\n",
       "      <td>-0.027902</td>\n",
       "      <td>-0.071123</td>\n",
       "      <td>-0.000668</td>\n",
       "      <td>albef14m_n</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>concapalbef14m_n</th>\n",
       "      <td>-0.033901</td>\n",
       "      <td>-0.076007</td>\n",
       "      <td>-0.006907</td>\n",
       "      <td>concapalbef14m_n</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>blip</th>\n",
       "      <td>-0.019671</td>\n",
       "      <td>-0.016178</td>\n",
       "      <td>-0.034759</td>\n",
       "      <td>blip</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>blip_n</th>\n",
       "      <td>-0.016323</td>\n",
       "      <td>-0.016789</td>\n",
       "      <td>-0.032531</td>\n",
       "      <td>blip_n</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "                         M         F         N             model\n",
       "lxmert           -0.029297 -0.068376  0.004679            lxmert\n",
       "lxmert_n         -0.019950 -0.051893  0.000668          lxmert_n\n",
       "concaplxmert     -0.020508 -0.064713 -0.037433      concaplxmert\n",
       "concaplxmert_n   -0.020787 -0.074481 -0.023619    concaplxmert_n\n",
       "albef4m          -0.020787 -0.068071  0.000446           albef4m\n",
       "albef4m_n        -0.025530 -0.074786 -0.006907         albef4m_n\n",
       "concapalbef4m_n  -0.023996 -0.065934 -0.008467   concapalbef4m_n\n",
       "albef14m         -0.034738 -0.074786  0.000446          albef14m\n",
       "albef14m_n       -0.027902 -0.071123 -0.000668        albef14m_n\n",
       "concapalbef14m_n -0.033901 -0.076007 -0.006907  concapalbef14m_n\n",
       "blip             -0.019671 -0.016178 -0.034759              blip\n",
       "blip_n           -0.016323 -0.016789 -0.032531            blip_n"
      ]
     },
     "execution_count": 5,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Define the data\n",
    "lxmert=[-0.02929688,-0.06837607, 0.00467914]\n",
    "lxmert_n=[-0.01994978,-0.05189255,0.00066845]\n",
    "concaplxmert=[-0.02050781,-0.06471306,-0.03743316]\n",
    "concaplxmert_n=[-0.02078683,-0.07448107,-0.02361854]\n",
    "albef_4m=[-0.02078683,-0.06807082,0.00044563]\n",
    "albef_4m_coco=[-0.02553013,-0.07478632,-0.00690731]\n",
    "albef_4m_cc3m=[-0.02399554,-0.06593407,-0.00846702]\n",
    "albef_14m=[-0.03473772,-0.07478632,0.00044563]\n",
    "albef_14m_coco=[-0.02790179,-0.07112332,-0.00066845]\n",
    "albef_14m_cc3m=[-0.03390067,-0.07600733,-0.00690731]\n",
    "blip=[-0.01967076,-0.01617827,-0.03475936]\n",
    "blip_coco=[-0.01632254,-0.01678877,-0.03253119]\n",
    "\n",
    "data_ = [lxmert, lxmert_n,\n",
    "         concaplxmert, concaplxmert_n,\n",
    "         albef_4m, albef_4m_coco, albef_4m_cc3m,\n",
    "         albef_14m, albef_14m_coco, albef_14m_cc3m,\n",
    "         blip, blip_coco]\n",
    "index_ = [\"lxmert\", \"lxmert_n\",\n",
    "          \"concaplxmert\", \"concaplxmert_n\",\n",
    "          \"albef4m\", \"albef4m_n\", \"concapalbef4m_n\", \n",
    "          \"albef14m\", \"albef14m_n\", \"concapalbef14m_n\", \n",
    "          \"blip\", \"blip_n\"]\n",
    "df = pd.DataFrame(data=data_, \n",
    "                  index=index_,\n",
    "                  columns=[\"M\",\"F\",\"N\"])\n",
    "df[\"model\"] = df.index\n",
    "df"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "id": "9bb67695",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<matplotlib.legend.Legend at 0x7f9a506b6040>"
      ]
     },
     "execution_count": 6,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 576x432 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "# Plot\n",
    "\n",
    "name=\"vqa_dba\"\n",
    "\n",
    "data_ = [\n",
    "         lxmert, lxmert_n,\n",
    "         albef_14m, albef_14m_coco,\n",
    "         blip, blip_coco]\n",
    "index_ = [\"lxmert\", \"lxmert_n\",\n",
    "          \"albef14m\", \"albef14m_n\", \n",
    "          \"blip\", \"blip_n\"]\n",
    "df = pd.DataFrame(data=data_, \n",
    "                  index=index_,\n",
    "                  columns=[\"M\",\"F\",\"N\"])\n",
    "df[\"model\"] = df.index\n",
    "\n",
    "# fig = plt.figure(figsize=(8, 10), )\n",
    "# ax0 = fig.add_subplot(grid[0, 0]) \n",
    "f, ax0 = plt.subplots(figsize=(8,6), tight_layout=True)\n",
    "# f.tight_layout(pad=0) \n",
    "\n",
    "bar_width = 2./len(models)\n",
    "\n",
    "cm = plt.get_cmap('Set3') \n",
    "colors = cm.colors\n",
    "bcm = plt.get_cmap('Pastel2')\n",
    "bcolors = bcm.colors\n",
    "ax0.set_xlim(-0.5, 5.5+bar_width) \n",
    "ax0.axvspan(-0.5, 1.5+bar_width, facecolor=bcolors[0], alpha=0.2)\n",
    "ax0.axvspan(1.5+bar_width, 3.5+bar_width, facecolor=bcolors[1], alpha=0.3)\n",
    "ax0.axvspan(3.5+bar_width, 5.5+bar_width, facecolor=bcolors[2], alpha=0.2)\n",
    "\n",
    "ax0.set_xticks([0.5+bar_width, 2.5+bar_width, 4.5+bar_width])\n",
    "# ax0.set_xticklabels(['$\\\\mathrm{\\\\textsc{LXMERT}_{CC3M}}$', '$\\\\mathrm{\\\\textsc{ALBEF}}$', '$\\\\mathrm{\\\\textsc{BLIP}}$'], fontsize=18)\n",
    "ax0.set_xticklabels(['$\\\\mathrm{\\\\textsc{LXMERT}_{3M}}$', \n",
    "                     '$\\\\mathrm{\\\\textsc{ALBEF}_{4M}}$', \n",
    "                     '$\\\\mathrm{\\\\textsc{ALBEF}_{14M}}$'], fontsize=18)\n",
    "\n",
    "plot_dict = {\n",
    "    'M': colors[0],\n",
    "    'F': colors[3],\n",
    "    'N': colors[1]}\n",
    "row_names = list(plot_dict.keys())\n",
    "\n",
    "X = np.array(range(len(df)))\n",
    "xs = [x[i] for i in range(3) for x in df.values]\n",
    "\n",
    "for i, name in list(enumerate([y for x in row_names for y in x])):\n",
    "    ys = [x for x in df[name].values]    \n",
    "    label=None\n",
    "    ax0.bar(X + i*bar_width, ys, \n",
    "            width=bar_width, label=label, color=plot_dict[name], \n",
    "            hatch=[None, \"//\"]*int(len(df)/2), \n",
    "            edgecolor='black') \n",
    "        \n",
    "\n",
    "ax0.set_ylabel('$\\\\mathrm{BiasAmp_{ A\\\\rightarrow{T}}}$', fontsize=20)\n",
    "ax0.set_ylim(min(xs)-0.003, max(xs)+0.002)\n",
    "\n",
    "ax0.tick_params(axis=\"y\", labelsize=20)\n",
    "ax0.grid(color='black', alpha=0.2, axis='y')\n",
    "\n",
    "ax0.spines['right'].set_visible(False)\n",
    "ax0.spines['top'].set_visible(False)\n",
    "\n",
    "h = [Patch(facecolor=colors[0], edgecolor='k', label='male'),\n",
    "     Patch(facecolor=colors[3], edgecolor='k', label='female'),\n",
    "     Patch(facecolor=colors[1], edgecolor='k', label='neutral'),\n",
    "     Patch(facecolor='#d6d6d6', edgecolor='#d6d6d6', label='$\\\\mathrm{M}$'),\n",
    "     Patch(facecolor='white', edgecolor='black', hatch='//',linewidth=0.1, label='$\\\\mathrm{M^{N}}$'),\n",
    "    ]\n",
    "ax0.legend(handles=h, ncol=1, scatterpoints=1, fontsize=14, loc='lower right')\n",
    "\n",
    "plt.savefig(f'figures/vqa_dba_hist_cc3m.pdf', bbox_inches='tight')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "20a7b136",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.7"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
